<a href="https://colab.research.google.com/github/swiminthewind/colab/blob/main/231122-%E6%B5%8B%E8%AF%95python%E7%89%88%E4%BA%94%E9%98%B6%E8%B0%90%E6%B3%A2%E7%9A%84%E6%AD%A3%E7%A1%AE%E6%80%A7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

In [2]:
import pandas as pd
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

In [3]:
roi = ee.FeatureCollection("projects/ee-grn/assets/cd_plain_cd")
csPlus= ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")
s2sr = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

In [4]:
timeField = 'system:time_start'
QA_BAND = 'cs'
CLEAR_THRESHOLD = 0.6

def cloud_remove(img):
    return img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)).select('.*').copyProperties(img, ['system:time_start'])

composite = s2sr.filterBounds(roi).filterDate('2022-01-01','2023-11-22').linkCollection(csPlus,[QA_BAND]).map(cloud_remove)

def s2addVariables(image):
  date = ee.Date(image.get(timeField))
  years = date.difference(ee.Date('1970-01-01'), 'year')
  return image.addBands(image.normalizedDifference(['B8', 'B4']).rename('NDVI')).addBands(ee.Image(years).rename('t')).float().addBands(ee.Image.constant(1))

filteredSentinel = composite.map(s2addVariables)
print(filteredSentinel.size().getInfo())

1245


In [5]:
dataset = filteredSentinel.select(['NDVI','t','constant'])
independents = ee.List(['constant', 't'])
dependent = ee.String('NDVI')

In [12]:
# 二阶谐波
harmonicIndependents_5 = ee.List(['constant', 't', 'cos1', 'sin1', 'cos2', 'sin2', 'cos3', 'sin3', 'cos4', 'sin4', 'cos5', 'sin5'])

def add_sincos_5(image):
  timeRadians1 = image.select('t').multiply(2 * math.pi)
  timeRadians2 = image.select('t').multiply(4 * math.pi)
  timeRadians3 = image.select('t').multiply(6 * math.pi)
  timeRadians4 = image.select('t').multiply(8 * math.pi)
  timeRadians5 = image.select('t').multiply(10 * math.pi)
  return image.addBands(timeRadians1.cos().rename('cos1')).addBands(timeRadians1.sin().rename('sin1')).addBands(timeRadians2.cos().rename('cos2')).addBands(timeRadians2.sin().rename('sin2')).addBands(timeRadians3.cos().rename('cos3')).addBands(timeRadians3.sin().rename('sin3')).addBands(timeRadians4.cos().rename('cos4')).addBands(timeRadians4.sin().rename('sin4')).addBands(timeRadians5.cos().rename('cos5')).addBands(timeRadians5.sin().rename('sin5'))

harmonicSentinel_5 = dataset.map(add_sincos_5)

harmonicTrend_5 = harmonicSentinel_5.select(harmonicIndependents_5.add(dependent)).reduce(ee.Reducer.linearRegression(numX=harmonicIndependents_5.length(),numY=1))

harmonicTrendCoefficients_5 = harmonicTrend_5.select('coefficients').arrayProject([0]).arrayFlatten([harmonicIndependents_5])

def fit_5(image):
  return image.addBands(image.select(harmonicIndependents_5).multiply(harmonicTrendCoefficients_5).reduce('sum').rename('fitted'))

fittedHarmonic_5 = harmonicSentinel_5.map(fit_5)

In [13]:
point = ee.Geometry.Point(103.69442853431933, 30.551562081391808)

rTable = ee.FeatureCollection(fittedHarmonic_5.filterBounds(point).map(lambda image: ee.Feature(
    None,
    {
        'date': image.date().format('yyyy-MM-dd'),
        'NDVI': image.reduceRegion(ee.Reducer.mean(), point, 10).get("NDVI"),
        'fitted': image.reduceRegion(ee.Reducer.mean(), point, 10).get("fitted")
    }
)))

rTable_dict = rTable.getInfo()

dates = [feat['properties']['date'] for feat in rTable_dict['features']]
ndvi_values = [feat['properties']['NDVI'] for feat in rTable_dict['features']]
fitted_values = [feat['properties']['fitted'] for feat in rTable_dict['features']]

df = pd.DataFrame({'Date': dates, 'NDVI': ndvi_values, 'Fitted': fitted_values})

df = df.dropna()

print(df)

           Date      NDVI    Fitted
0    2022-01-01  0.480345  0.571750
11   2022-01-29  0.638256  0.636678
17   2022-02-13  0.727610  0.668168
25   2022-03-05  0.589392  0.678349
26   2022-03-07  0.687620  0.679440
27   2022-03-10  0.715548  0.681192
29   2022-03-15  0.706660  0.683665
30   2022-03-17  0.713792  0.684129
39   2022-04-09  0.631566  0.602623
43   2022-04-19  0.521598  0.488733
44   2022-04-21  0.573421  0.461212
46   2022-04-26  0.480552  0.388633
49   2022-05-04  0.220395  0.275057
51   2022-05-09  0.178203  0.216474
54   2022-05-16  0.197687  0.164504
71   2022-06-28  0.618742  0.626153
74   2022-07-05  0.619651  0.701372
75   2022-07-08  0.668581  0.724430
81   2022-07-23  0.735616  0.767988
82   2022-07-25  0.686726  0.767587
83   2022-07-28  0.771458  0.765650
87   2022-08-07  0.763696  0.754500
88   2022-08-09  0.751144  0.752132
89   2022-08-12  0.808487  0.748506
93   2022-08-22  0.608793  0.731786
99   2022-09-06  0.694187  0.657774
117  2022-10-21  0.278708  0